In [4]:
import pydot
import matplotlib.pyplot as plt
import time
#import math
#import networkx as nx
#import random

# Step 1

## Uploading the data

In [5]:
def get_data(dot_file_path):
     #= 'Datasets/LesMiserables.dot'
    with open(dot_file_path, 'r') as dot_file:
        dot_data = dot_file.read()
    graph = pydot.graph_from_dot_data(dot_data)

    nodes = graph[0].get_nodes()
    edges = graph[0].get_edges()
    
    node_list = [node.get_name() for node in nodes]
    edge_list = [(edge.get_source(), edge.get_destination()) for edge in edges]
    name= str(list(edges[0].get_attributes().keys())[0])
    weights = {(edge.get_source(), edge.get_destination()):float(edge.get_attributes()[name]) for edge in edges}

    return nodes,edges,node_list,edge_list,weights
#graget_data('Datasets/LesMiserables.dot')
#nodes,edges,node_list,edge_list,weights = get_data('Datasets/LesMiserables.dot')
#print(('1','40') in edge_list)
# print(len(node_list)**3) #nodes
# print(len(edge_list))
# print(len(edge_list)*len(node_list)) #edges
# print()
# Nodes:  77 
#Edges:  254
#Total:  33982

## Tree Layout

### Counting the number of children/parents for each node

In [15]:
#COUNTING THE NUMBER OF CHILD NODES FOR EACH NODE
def f_childs(file_name_name):   
    nodes = get_data(file_name_name)[0]
    edges = get_data(file_name_name)[1]
    node_child_count = {node.get_name(): 0 for node in nodes}
    node_childs = {node.get_name(): [] for node in nodes}
    for edge in edges:
        source = edge.get_source()
        destination = edge.get_destination()
        if source in node_child_count:
            node_child_count[source] += 1 
            node_childs[source].append(destination) #comment
    return node_child_count, node_childs

#Count
#print(f_childs('Datasets/LesMiserables.dot')[0])
#print(sum(list(f_childs('Datasets/LesMiserables.dot')[0].values())))
#child_11,child_25,child_26 = f_childs('Datasets/LesMiserables.dot')[1]['11'],f_childs('Datasets/LesMiserables.dot')[1]['25'],f_childs('Datasets/LesMiserables.dot')[1]['26']
#print('CHILD')
#print('11: ',child_11)
#print('25: ',child_25)
#print('26": ',child_26)

#List      
#print(f_childs('Datasets/LesMiserables.dot')[1])

def f_parents(file_name_name):
    nodes = get_data(file_name_name)[0]
    edges = get_data(file_name_name)[1]
    node_parent_count = {node.get_name(): 0 for node in nodes}
    node_parents = {node.get_name(): [] for node in nodes}
    for edge in edges:
        source = edge.get_source()
        destination = edge.get_destination()
        if destination in node_parent_count:
            node_parent_count[destination] += 1 
            node_parents[destination].append(source)
    return node_parent_count, node_parents

#Count
#print(f_parents('Datasets/LesMiserables.dot')[0]['11'])
#parent_27 = f_parents('Datasets/LesMiserables.dot')[1]['27']
#print("PARENT")
#print('27: ', parent_27)
#List      
#print(f_parents('Datasets/LesMiserables.dot')[1])

def f_edges(file_name):
    nodes = get_data(file_name)[0]
    edges = get_data(file_name)[1]
    node_edges_count = {node.get_name(): 0 for node in nodes}
    node_edges = {node.get_name(): [] for node in nodes}
    
    parents = f_parents(file_name)[1]
    childs = f_childs(file_name)[1]

    for node,l_parents in parents.items():
        for parent in l_parents:
            node_edges_count[node] +=1
            node_edges[node].append(parent)
    
    for node,l_childs in childs.items():
        for child in l_childs:
            node_edges_count[node] +=1
            node_edges[node].append(child)
    return node_edges_count,node_edges

#print(f_edges('Datasets/LesMiserables.dot')[1])
#print(f_parents('Datasets/LesMiserables.dot')[1])

#edges = f_edges('Datasets/LesMiserables.dot')
#parents = f_parents('Datasets/LesMiserables.dot')

#print(edges == parents)
#print(edges[0])

{'1': [], '2': ['1'], '3': ['2'], '4': ['2', '3'], '5': ['2'], '6': ['2'], '7': ['2'], '8': ['2'], '9': ['2'], '10': ['2'], '11': ['2', '3', '4'], '12': ['11'], '13': ['11'], '14': ['11'], '15': ['11'], '16': ['11'], '24': ['11', '13'], '25': ['11', '24'], '26': ['11', '24', '25'], '27': ['11', '25', '26'], '28': ['11', '24', '25', '26', '27'], '29': ['11', '28'], '30': ['11', '24', '28'], '32': ['11', '24', '28'], '33': ['11'], '34': ['11', '28'], '35': ['11', '30'], '36': ['11', '30', '35'], '37': ['11', '30', '35', '36'], '38': ['11', '30', '35', '36', '37'], '39': ['11', '30', '35', '36', '37', '38'], '44': ['11', '27', '28'], '45': ['11', '29'], '49': ['11', '26', '28'], '50': ['11', '27'], '52': ['11', '27', '50'], '56': ['11', '26', '27', '49', '50', '52'], '59': ['11', '28', '49', '56'], '65': ['11', '49', '56', '59'], '69': ['11', '25', '26', '28', '49'], '70': ['11', '25', '26', '28', '49', '69'], '71': ['11', '25', '26', '28', '59', '69', '70'], '72': ['11', '26', '28', '49'

### Organizing the nodes throughout the graph, getting their coordinates for the tree layout

In [7]:
def tree_layout(file):
    total = 0
    n=0
    graph = get_data(file)
    nodes = graph[0]
    edges = graph[1]
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes= []
    node_child_count = f_childs(file)[0]
    node_childs = f_childs(file)[1]
    #lonely_nodes_y = -4
    
    for edge in edges:
        n+=1
        l_m = []
        m=0
        #print(edge)#.get_source())
    #    print(node_childs[str(edge.get_destination())])
        parent_x = node_positions[str(edge.get_source())][0]
        parent_y = node_positions[str(edge.get_source())][1]
        step = 3#random.choice([2,3])

        #nnodes with only one child
        if len(node_childs[str(edge.get_source())]) == 1 and int(edge.get_destination()) not in done_nodes:# and int(edge.get_destination()) not in done_nodes:
            #print(edge)
            #random.choice([2,3])
            child_x = parent_x + step
            child_y = parent_y
            node_positions[edge.get_destination()] = (child_x, child_y)
            done_nodes.append(int(edge.get_destination()))
        
        #nodes with more than one node
        elif len(node_childs[str(edge.get_source())]) > 1 and int(edge.get_destination()) not in done_nodes:
            if len(node_childs[str(edge.get_source())])%2 != 0:
                max_y = (node_child_count[str(edge.get_source())]-1)*3
            else:
                max_y = (node_child_count[str(edge.get_source())]-1)*3+1
            for child in node_childs[str(edge.get_source())]:
                m+=1
                child_x = parent_x + step
                child_y = parent_y + max_y
                node_positions[child] = (child_x, child_y)
                max_y -=8
                done_nodes.append(int(edge.get_destination()))
            l_m.append(m)
            total += m
        total+=n
        #elif len(node_childs[str(edge.get_source())]) == 0 and len(node_childs[str(edge.get_destination())]) == 0:
        #    node_positions[child]

        #elif len([done_node for done_node in done_nodes if done_node == edge.get_destination()])==1:
        #    child_x += 1               
        
    return node_positions,n,l_m,total

#pos,n,m,total = tree_layout('Datasets/LesMiserables.dot')
#print("Edges: ",n, '\n',"Nodes: ",m, '\n',"Total: ",total,)
#f_childs(nodes,edges)[1]['17']

### Improved version of the Tree layout

Get the nodes that have the same parent and are directly connected to each other

In [8]:
def common_parents(file_name):
    nodes,edges,node_list,edge_list,weights = get_data(file_name)
    ans = {node:[] for node in node_list}
    count = {node:0 for node in node_list}
    for edge1 in edges:
        for edge2 in edges:
            if edge1.get_source() == edge2.get_source() and edge1.get_destination() != edge2.get_destination() and (str(edge1.get_destination()),str(edge2.get_destination())) in edge_list:# and str(edge1.get_destination()) not in ans:
                #ans.append((str(edge1.get_destination()),str(edge2.get_destination())))
                #ans[edge1.get_source()] = []
                ans[edge1.get_source()].append((str(edge1.get_destination()),str(edge2.get_destination())))
   
    for node,edges in ans.items():
        for edge in edges:
            #print(edge)
            count[edge[1]] += 1
    return count

#common_parents('Datasets/LesMiserables.dot')

In [9]:
def tree_layout_improv(file):
    #returns the coordinates for each node 
    graph = get_data(file)
    nodes = graph[0]
    edges = graph[1]
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes= []
    node_child_count = f_childs(file)[0]
    node_childs = f_childs(file)[1]
    
    for edge in edges:
        parent_x = node_positions[str(edge.get_source())][0]
        parent_y = node_positions[str(edge.get_source())][1]
        
        #weights
        position = str(str(edge.get_attributes())).rfind(':')
        if int(position) != -1:
            w = int(str(edge.get_attributes())[position+3:-2])
            if w != 1:
                w = w/10

        #nodes with only one child
        if len(node_childs[str(edge.get_source())]) == 1 and int(edge.get_destination()) not in done_nodes:# and int(edge.get_destination()) not in done_nodes:
            #print(edge)
            step = 3#random.choice([2,3])
            child_x = parent_x + step
            child_y = parent_y
            node_positions[edge.get_destination()] = (child_x, child_y)
            done_nodes.append(int(edge.get_destination()))
            print()
        
        #nodes with more than one node
        elif len(node_childs[str(edge.get_source())]) > 1 and int(edge.get_destination()) not in done_nodes:
            if len(node_childs[str(edge.get_source())])%2 != 0:
                max_y = (node_child_count[str(edge.get_source())]-1)*3
            else:
                max_y = (node_child_count[str(edge.get_source())]-1)*3+1
            for child in node_childs[str(edge.get_source())]:
                step = 3#random.choice([2,3])
                child_x = parent_x + step
                child_y = parent_y + max_y
                node_positions[child] = (child_x, child_y)
                max_y -=8
                done_nodes.append(int(edge.get_destination()))
        #elif len([done_node for done_node in done_nodes if done_node == edge.get_destination()])==1:
        #    child_x += 1
                
    #correcting the child_x in cases where sibling nodes are connected to each other
    for node,count in [(node,count) for node,count in common_parents(file).items() if count != 0]:
        node_positions[node] = (node_positions[node][0]+count/3,node_positions[node][1])
    
    return node_positions

#tree_layout_improv('Datasets/LesMiserables.dot')

### Time complexity of the improved version

In [10]:
 # Record the start time

# Code to execute the algorithm
# For example:
def time_complexity(file,layout,ite):
    ans = []
    for i in range(ite):
        if layout == 'tree_layout':
            start_time = time.time() 
            result = tree_layout(file)
            end_time = time.time() 
        
        if layout == 'tree_layout_improv':
            start_time = time.time() 
            result = tree_layout_improv(file)
            end_time = time.time() 
         # Record the end time
        elapsed_time = end_time - start_time  # Calculate the elapsed time
        ans.append(elapsed_time)
    print('In average, the layout',layout, 'takes', round(sum(ans)/len(ans), 2),'seconds to implement on the', file,'dataset.') 

#time_complexity('Datasets/LesMiserables.dot','tree_layout_improv',10)

### Plot the graph

In [11]:
#HOW TO PLOT A GENERAL GRAPH
def show_graph(file,layout,dims):
    if layout == 'tree_layout':
        node_positions = tree_layout(file)
        
    if layout == 'tree_layout_improv':
        node_positions = tree_layout_improv(file)

    if layout == 'BFS_tree':
        node_positions = BFS_tree(file)

    nodes,edges = get_data(file)[0],get_data(file)[1]
    plt.figure(figsize=(dims[0], dims[1]))

    for edge in edges:
        start_node = edge.get_source()
        end_node = edge.get_destination()
        start_pos = node_positions[start_node]
        end_pos = node_positions[end_node]

        position = str(str(edge.get_attributes())).rfind(':')
        if int(position) != -1:
            w = int(str(edge.get_attributes())[position+3:-2])
            if w != 1:
                w = w/10
        #else:
        #    w=1
        #plt.scatter([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], color='black', s=10, linewidths=w)
        plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 

    for node, pos in node_positions.items():
        plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # Use marker='o' for circle markers
        plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
    plt.axis('off')
    plt.gca().set_facecolor('w')
    plt.savefig(layout+'plot.png')
    plt.show()
    
#show_graph('Datasets/LesMiserables.dot','tree_layout1',(18,18))
#show_graph('Datasets/LesMiserables.dot','tree_layout_improv',(18,18))

### Bonus Task

In [12]:
#HOW TO CREATE THE DEFAULT LAYOUT
def bonus_task(file,layout):
    with open(file, 'r') as dot_file:
        dot_data = dot_file.read()

    # Create a graph object from the .dot data
    graph = pydot.graph_from_dot_data(dot_data)

    # Set the layout algorithm to "dot" (or any other supported layout type)
    graph[0].set_graph_defaults(layout=layout)
    graph[0].set_graph_defaults(layout=layout,)
    # Render the graph with Graphviz
    output_file_path = layout+'.png'
    graph[0].write_png(output_file_path)


#for layout in ["dot", "neato", "circo", "fdp", "twopi"]:
#    bonus_task('Datasets/LesMiserables.dot',layout)
#bonus_task('Datasets/LesMiserables.dot',"twopi")
#print("Graph with dot layout saved as:", output_file_path)


# Step 2

### Constructing a BFS tree

In [13]:
def BFS_tree(file):
    total = 0
    n=0
    graph = get_data(file)
    nodes = graph[0]
    edges = graph[1]
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes = set()
    node_child_count = f_childs(file)[0]
    node_childs = f_childs(file)[1]
    #lonely_nodes_y = -4
    min_y = 0

    for node in nodes:#node_childs[str(node)]:
        done_nodes.add(str(node.get_name()))
        childs = node_childs[str(node.get_name())]
        parent_x = node_positions[str(node.get_name())][0]
        parent_y = node_positions[str(node.get_name())][1]
        step = 3

        #print('Number of childs: ',len(node_childs[str(node.get_name())]))
        #print('node: ',str(node.get_name()))
        if len(node_childs[str(node.get_name())]) == 1 and str(node_childs[str(node.get_name())][0]) not in done_nodes:# and int(edge.get_destination()) not in done_nodes:
            #print('child: ',str(node_childs[str(node.get_name())][0]))
            #random.choice([2,3])
            child_x = parent_x + step
            if parent_y > min_y:
                child_y = min_y-6
            else:
                child_y = parent_y
            node_positions[str(node_childs[str(node.get_name())][0])] = (child_x, child_y)
            done_nodes.add(str(node_childs[str(node.get_name())][0]))
            min_y = child_y

        elif len(node_childs[str(node.get_name())]) >1:# and node_childs[str(node.get_name())] not in done_nodes:
            #if len(node_childs[str(node.get_name())])%2 != 0:
            
            #else:
            #    max_y = (node_child_count[str(node.get_name())]-1)*3+1
            #print('node: ',node.get_name(),'parent_y: ', parent_y)
            new_childs = len([child for child in node_childs[str(node.get_name())] if child not in done_nodes])
            max_y = (new_childs-1)*3
            #print('node: ',node.get_name(),'node_position: ',node_positions[node.get_name()],'max_y',max_y)
            
            done_childs = []
            for child in [new_child for new_child in node_childs[str(node.get_name())] if new_child not in done_nodes]:
                done_childs.append(child)
                #if str(child) not in done_nodes:
                child_x = parent_x + step
                if parent_y + max_y > min_y:
                    child_y = min_y-6+max_y
                else:
                    child_y = parent_y + max_y
                
                #child_y = parent_y + max_y
                node_positions[child] = (child_x, child_y)
                done_nodes.add(str(child))
                max_y -= 6
            if len(done_childs)>0:
                min_y = node_positions[done_childs[-1]][1]
            #print(done_childs)
            #min_y = node_positions[[child for child in node_childs[str(node.get_name())] if child not in done_nodes][-1]]
            #print('min_y from the childs of node', node,':',min_y)
        print(min_y) 
        #print('node: ',node,'node position: ', node_positions[node.get_name()],'done nodes: ',done_nodes) 
        
    # plt.figure(figsize=(18, 18))

    # reached_dests = set()
    # for edge in edges:
    #     #print('source: ',node_positions[str(edge.get_source())],'dest: ', node_positions[str(edge.get_destination())])
    #     if node_positions[str(edge.get_source())][0] != node_positions[str(edge.get_destination())][0] and str(edge.get_destination()) not in reached_dests:
    #         start_node = edge.get_source()
    #         end_node = edge.get_destination()
    #         start_pos = node_positions[start_node]
    #         end_pos = node_positions[end_node]

    #         position = str(str(edge.get_attributes())).rfind(':')
    #         if int(position) != -1:
    #             w = int(str(edge.get_attributes())[position+3:-2])
    #             if w != 1:
    #                 w = w/10
    #     #else:
    #     #    w=1
    #     #plt.scatter([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], color='black', s=10, linewidths=w)
    #     plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 
    #     reached_dests.add(str(edge.get_destination()))
    # for node, pos in node_positions.items():
    #     plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # Use marker='o' for circle markers
    #     plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
    # plt.axis('off')
    # plt.gca().set_facecolor('w')
    # plt.savefig('BFS_tree_plot.png')
    # plt.show()
        
    return node_positions

#tree = BFS_tree('Datasets/LesMiserables.dot')
#tree
#show_graph('Datasets/LesMiserables.dot','BFS_tree',(18,18))
# #print("Edges: ",n, '\n',"Nodes: ",m, '\n',"Total: ",total,)
# #f_childs(nodes,edges)[1]['17']

In [ ]:
#print('chids 11: ', f_childs('Datasets/LesMiserables.dot')[0]['11'])
#11 has 33 childs
# print('29:', tree['29'])
print('31:', tree['31'])
#f_childs('Datasets/LesMiserables.dot')[1]['29']


In [ ]:
# childs27 = f_childs('Datasets/LesMiserables.dot')[1]['27']
# print('27 childs',childs27)
# print()
# for child in childs27:
#     print(child,'parents',f_parents('Datasets/LesMiserables.dot')[1][str(child)])


# childs =  f_childs('Datasets/LesMiserables.dot')[1]['11']
# nodes = ['12','13','14','15','16','24']
#for i in childs11:
 #   print(i,f_childs('Datasets/LesMiserables.dot')[1][i])


parents46 = f_parents('Datasets/LesMiserables.dot')[1]['48']
parents46


In [ ]:
tree = BFS_tree('Datasets/LesMiserables.dot')
with open('output.txt', 'w') as file:    
    for node,coordinates in tree.items():
        line = str(node)+' : '+str(coordinates)+'\n'
        file.write(str(line))

In [ ]:
#     for edge in edges:
#         n+=1
#         l_m = []
#         m=0
#         #print(edge)#.get_source())
#     #    print(node_childs[str(edge.get_destination())])
#         parent_x = node_positions[str(edge.get_source())][0]
#         parent_y = node_positions[str(edge.get_source())][1]
#         step = 3#random.choice([2,3])

#         #nnodes with only one child
#         if len(node_childs[str(edge.get_source())]) == 1 and int(edge.get_destination()) not in done_nodes:# and int(edge.get_destination()) not in done_nodes:
#             #print(edge)
#             #random.choice([2,3])
#             child_x = parent_x + step
#             child_y = parent_y
#             node_positions[edge.get_destination()] = (child_x, child_y)
#             done_nodes.append(int(edge.get_destination()))
        
#         #nodes with more than one node
#         elif len(node_childs[str(edge.get_source())]) > 1 and int(edge.get_destination()) not in done_nodes:
#             if len(node_childs[str(edge.get_source())])%2 != 0:
#                 max_y = (node_child_count[str(edge.get_source())]-1)*3
#             else:
#                 max_y = (node_child_count[str(edge.get_source())]-1)*3+1
#             for child in node_childs[str(edge.get_source())]:
#                 m+=1
#                 child_x = parent_x + step
#                 child_y = parent_y + max_y
#                 node_positions[child] = (child_x, child_y)
#                 max_y -=8
#                 done_nodes.append(int(edge.get_destination()))
#             l_m.append(m)
#             total += m
#         total+=n
#         #elif len(node_childs[str(edge.get_source())]) == 0 and len(node_childs[str(edge.get_destination())]) == 0:
#         #    node_positions[child]

#         #elif len([done_node for done_node in done_nodes if done_node == edge.get_destination()])==1:
#         #    child_x += 1               
        
#     return node_positions,n,l_m,total

# Radial Layout

### Sorting the nodes based on their degree

In [ ]:
# def f_edges(file_name):
#     nodes = get_data(file_name)[0]
#     edges = get_data(file_name)[1]
#     node_edges_count = {node.get_name(): 0 for node in nodes}
#     node_edges = {node.get_name(): [] for node in nodes}
    
#     parents = f_parents(file_name)[1]
#     childs = f_childs(file_name)[1]

#     for node,l_parents in parents.items():
#         for parent in l_parents:
#             node_edges_count[node] +=1
#             node_edges[node].append(parent)
    
#     for node,l_childs in childs.items():
#         for child in l_childs:
#             node_edges_count[node] +=1
#             node_edges[node].append(child)
#     return node_edges_count,node_edges

In [26]:
def sort_nodes_degree(file):
    return dict(sorted(f_edges(file)[1].items(), key=lambda item: len(item[1]), reverse=True))

#print(sort_nodes_degree('Datasets/LesMiserables.dot'))

{'11': ['2', '3', '4', '12', '13', '14', '15', '16', '24', '25', '26', '27', '28', '29', '30', '32', '33', '34', '35', '36', '37', '38', '39', '44', '45', '49', '50', '52', '56', '59', '65', '69', '70', '71', '72', '73'], '49': ['11', '26', '28', '47', '56', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '69', '70', '72', '74', '75', '76', '77'], '56': ['11', '26', '27', '49', '50', '52', '18', '40', '42', '55', '57', '58', '59', '60', '62', '63', '64', '65', '66'], '28': ['11', '24', '25', '26', '27', '29', '30', '32', '34', '44', '49', '59', '69', '70', '71', '72', '73'], '26': ['11', '24', '25', '27', '28', '40', '41', '42', '43', '49', '56', '69', '70', '71', '72', '76'], '24': ['11', '13', '17', '18', '19', '20', '21', '22', '23', '25', '26', '28', '30', '31', '32'], '59': ['11', '28', '49', '56', '58', '60', '61', '62', '63', '64', '65', '66', '67', '71', '77'], '65': ['11', '49', '56', '59', '58', '60', '61', '62', '63', '64', '66', '67', '77'], '63': ['49', '56', '

In [ ]:
def tree(file):
    graph = get_data(file)
    nodes = graph[0]
    edges = graph[1]

    tree = {node.get_name(): [] for node in nodes}
    sorted_nodes = sort_nodes_degree(file)
    done_nodes = set()
    for node,dests in sorted_nodes.items():
        if node not in done_nodes:
            done_nodes.add(node) 
            if len(dests)==1 and dests[0] not in done_nodes:
                tree[node].append(dests[0])

            else:   
                for dest in dests:
                    if dest not in done_nodes:
                        done_nodes.add(dest)
                        return tree()




print(tree('Datasets/LesMiserables.dot'))


# def order_radial(file):
#     tree = {}
#     sorted_nodes = sort_nodes_degree(file)
#     done_nodes = set()
#     for node,childs in sorter_nodes.items():
#         tree[node]


In [ ]:
node_positions_radial = {node.get_name(): (0, 0) for node in nodes}
done_nodes= []
lonely_nodes_h = 4
radius = 2

for node in sorted_node_childs.items():
    #for edge in edges:
    #print(edge)#.get_source())
#    print(node_childs[str(edge.get_destination())])
    #for child in
    parent_x = node_positions[str(node[0])][0]
    parent_y = node_positions[str(node[0])][1]
    if int(node_child_count[str(node[0])]) > 1 and node not in done_nodes:#and int(edge.get_destination()) not in done_nodes:
        n_child = int(node_child_count[str(node[0])])
        #print(n_child)
        theta = math.pi/n_child
        angle = theta-theta/2
        for child in sorted_node_childs[node[0]]:
            #print(child)            
            child_x = parent_x + math.cos(angle)*sorted_node_childs[node[0]].index(child)
            child_y = parent_y + math.sin(angle)*sorted_node_childs[node[0]].index(child)
            node_positions_radial[node[0]] = (child_x,child_y)
    #print(node[0])
    #done_nodes.append(node[0])
        #l_parent = node_child_count[str(node[0])]
        #l_child = node_child_count[str(node[0])]
        #theta = min(l_child/(l_parent-1),2*math.acos(radius/(radius+2)))
        #for child in list(sorted_node_childs[node[0]]):
        #    child_x = 
            #print('parent: ', node[0],'child: ',child)
    
    if int(node_child_count[str(node[0])]) <= 1 and node not in done_nodes:
        child_x = parent_x+2
        child_y = parent_y+2
        node_positions_radial[node[0]] = (child_x,child_y)
    done_nodes.append(node[0])
#print(node_positions_radial['11'])
#print(done_nodes)

In [ ]:
#print(sorted_node_childs)
radius = 1
node_positions_radial = {node.get_name(): (0, 0) for node in nodes}
done_nodes= []
lonely_nodes_h = 4

for edge,node in zip(edges,sorted_node_childs):
    #print(edge)#.get_source())
#    print(node_childs[str(edge.get_destination())])
    parent_x = node_positions[str(edge.get_source())][0]
    parent_y = node_positions[str(edge.get_source())][1]
    if len(node_childs[str(edge.get_source())]) == 1 and int(edge.get_destination()) not in done_nodes:
        #print(edge)
        child_x = parent_x + 3
        child_y = parent_y
        node_positions[edge.get_destination()] = (child_x, child_y)
        done_nodes.append(int(edge.get_destination()))
    
    
    elif len(node_childs[str(edge.get_source())]) > 1 and int(edge.get_destination()) not in done_nodes:
        if len(node_childs[str(edge.get_source())])%2 != 0:
            max_y = (node_child_count[str(edge.get_source())]-1)*2
        else:
            max_y = (node_child_count[str(edge.get_source())]-1)*2+1
        for child in node_childs[str(edge.get_source())]:
            child_x = parent_x + 2
            child_y = parent_y + max_y
            node_positions[child] = (child_x, child_y)
            max_y -=5
            done_nodes.append(int(edge.get_destination()))











        num_points = node_child_count[edge.get_source()]
        angle_increment = 2 * math.pi / num_points
        x_coords = []
        y_coords = []

    # Calculate coordinates of each point
    for i in range(num_points):
        # Calculate angle for this point
        angle = i * angle_increment
        
        # Calculate coordinates using trigonometry
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        
        # Add coordinates to the lists
        x_coords.append(x)
        y_coords.append(y)
    
    radius += 2

In [ ]:
# Number of points
num_points = node_child_count[]

# Radius of the circle
radius = 1.0

# Calculate the angle between each point
angle_increment = 2 * math.pi / num_points

# Initialize lists to store the coordinates of the points
x_coords = []
y_coords = []

# Calculate coordinates of each point
for i in range(num_points):
    # Calculate angle for this point
    angle = i * angle_increment
    
    # Calculate coordinates using trigonometry
    x = radius * math.cos(angle)
    y = radius * math.sin(angle)
    
    # Add coordinates to the lists
    x_coords.append(x)
    y_coords.append(y)
for x,y in zip(x_coords,y_coords):
    print(x,y)
# Plot the points on a circle
plt.figure(figsize=(6, 6))
plt.scatter(x_coords, y_coords, color='blue', label='Points')
plt.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], color='red')  # Connect the last point to the first point
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Points on a Circle')
plt.axis('equal')  # Equal aspect ratio for x and y axes
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
#print(sorted_node_childs)
radius = 1
node_positions_radial = {node.get_name(): (0, 0) for node in nodes}

for node in sorted_node_childs:
    num_points = node_child_count[node]
    angle_increment = 2 * math.pi / num_points
    x_coords = []
    y_coords = []

    # Calculate coordinates of each point
    for i in range(num_points):
        # Calculate angle for this point
        angle = i * angle_increment
        
        # Calculate coordinates using trigonometry
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        
        # Add coordinates to the lists
        x_coords.append(x)
        y_coords.append(y)
    
    radius += 2

In [ ]:
#'Datasets/LesMiserables.dot'
import networkx as nx
import math

# Load the .dot file
dot_file_path = 'Datasets/LesMiserables.dot'

# Read the .dot file
G = nx.drawing.nx_pydot.read_dot(dot_file_path)

# Calculate node degrees
node_degrees = dict(G.degree())
node_positions = {node.get_name(): (0, 0) for node in nodes}

# Calculate radial positions based on node degrees
max_degree = max(node_degrees.values())
radius_scale = 10  # Adjust this value as needed
node_positions = {}
for node, degree in node_degrees.items():
    radius = degree * radius_scale / max_degree
    theta = 2 * math.pi * (node_degrees[node] / len(G))
    node_positions[node] = {'r': radius, 'theta': theta}

# Generate .dot file with calculated node positions
with open('radial_layout.dot', 'w') as dot_file:
    dot_file.write('graph {\n')
    for node, pos in node_positions.items():
        r = pos['r']
        theta = pos['theta']
        x = r * math.cos(theta)
        y = r * math.sin(theta)
        dot_file.write(f'    {node} [pos="{x},{y}!"];\n')
    
    # Write edges to the .dot file
    for edge in G.edges():
        dot_file.write(f'    {edge[0]} -- {edge[1]};\n')
    
    dot_file.write('}\n')

#plt.figure(figsize=(100, 100))
for node, pos in node_positions.items():
    x = pos['r'] * math.cos(pos['theta'])
    y = pos['r'] * math.sin(pos['theta'])
    plt.scatter(x, y, label=node, color='blue')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Radial Layout of Nodes')
plt.legend()
plt.grid(True)
plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('radial_plot.png')
plt.show()

    




In [ ]:
#print(sorted_node_childs)
radius = 1
node_positions_radial = {node.get_name(): (0, 0) for node in nodes}
def plot_radial_layout(nodes):
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.spines['polar'].set_visible(False)
    
    #plot node and edges
    for node,childs in nodes.items():
        if len(childs) != 0:
            num_points = node_child_count[node]
            theta = 2 * math.pi / num_points
            ax.plot(theta, radius, 'o') #plot node
            ax.text(theta, radius, str(node), ha='center', va='center', fontsize=8)
            # Draw edges
            for child in list(childs):
                i = childs.index(child)
                angle = i * angle_increment
                
                child_x, child_y = radius * math.cos(angle),radius * math.sin(angle)
                child_radius, child_theta = child_x, child_y
                ax.plot([theta, child_theta], [radius, child_radius], 'k-',linewidth =0.5)

    plt.show()

# Your existing code for node positions and child nodes

# Example usage
plot_radial_layout(sorted_node_childs)

In [ ]:
#print(sorted_node_childs)
radius = 1
node_positions_radial = {node.get_name(): (0, 0) for node in nodes}
def plot_radial_layout2(nodes):
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.spines['polar'].set_visible(False)
    
    #plot node and edges
    for node in nodes:
        n_childs = node_child_count[node]
        if int(n_childs) != 0:
            num_points = int(n_childs)
            theta = 2 * math.pi / num_points
            ax.plot(theta, radius, 'o') #plot node
            ax.text(theta, radius, str(node), ha='center', va='center', fontsize=8)
            x_parent = math.cos(theta)*radius
            y_parent = math.sin(theta)*radius
            # Draw edges
            for child in node_childs[node]:
                i = node_childs[node].index(child)
                angle = i * angle_increment
                
                child_x, child_y = radius * math.cos(angle),radius * math.sin(angle)
                child_radius, child_theta = child_x, child_y
                ax.plot([theta, child_theta], [radius, child_radius], 'k-',linewidth =0.5)

    plt.show()

# Your existing code for node positions and child nodes

# Example usage
plot_radial_layout2(sorted_node_childs)

### Trash-ish for tree layout

In [ ]:
node_positions = {node.get_name(): (0, 0) for node in nodes}
done_nodes= []
lonely_nodes_h = 4

for edge in edges:
    #print(edge)#.get_source())
#    print(node_childs[str(edge.get_destination())])
    parent_x = node_positions[str(edge.get_source())][0]
    parent_y = node_positions[str(edge.get_source())][1]
    if len(node_childs[str(edge.get_source())]) == 1:# and int(edge.get_destination()) not in done_nodes:
        #print(edge)
        child_x = parent_x + 3
        child_y = parent_y
        node_positions[edge.get_destination()] = (child_x, child_y)
        done_nodes.append(int(edge.get_destination()))
    
    
    elif len(node_childs[str(edge.get_source())]) > 1 and int(edge.get_destination()) not in done_nodes:
        if len(node_childs[str(edge.get_source())])%2 != 0:
            max_y = (node_child_count[str(edge.get_source())]-1)*2
        else:
            max_y = (node_child_count[str(edge.get_source())]-1)*2+1
        for child in node_childs[str(edge.get_source())]:
            child_x = parent_x + 2
            child_y = parent_y + max_y
            node_positions[child] = (child_x, child_y)
            max_y -=5
            done_nodes.append(int(edge.get_destination()))
print(node_positions)

In [ ]:
def tree_layout(file):
    graph = get_data(file)
    nodes = graph[0]
    edges = graph[1]
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes= []
    
    for edge in edges:
        #print(edge)#.get_source())
    #    print(node_childs[str(edge.get_destination())])
        parent_x = node_positions[str(edge.get_source())][0]
        parent_y = node_positions[str(edge.get_source())][1]
        node_child_count = f_childs(file)[0]
        node_childs = f_childs(file)[1]
        if len(node_childs[str(edge.get_source())]) == 1:# and int(edge.get_destination()) not in done_nodes:
            #print(edge)
            step = 3#random.choice([2,3])
            child_x = parent_x + step
            child_y = parent_y
            node_positions[edge.get_destination()] = (child_x, child_y)
            done_nodes.append(int(edge.get_destination()))
        
        
        elif len(node_childs[str(edge.get_source())]) > 1 and int(edge.get_destination()) not in done_nodes:
            if len(node_childs[str(edge.get_source())])%2 != 0:
                max_y = (node_child_count[str(edge.get_source())]-1)*3
            else:
                max_y = (node_child_count[str(edge.get_source())]-1)*3+1
            for child in node_childs[str(edge.get_source())]:
                step = 3#random.choice([2,3])
                child_x = parent_x + step
                child_y = parent_y + max_y
                node_positions[child] = (child_x, child_y)
                max_y -=8
                done_nodes.append(int(edge.get_destination()))
        
        elif len([done_node for done_node in done_nodes if done_node == edge.get_destination()])==1:
            child_x += 1
    return node_positions

print(tree_layout('Datasets/LesMiserables.dot'))
#f_childs(nodes,edges)[1]['17']

### Trash-ish for radial layout

In [ ]:
def plot_radial_layout11(nodes, center_x, center_y):
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.spines['polar'].set_visible(False)
    
    # Plot node and edges
    for node, childs in nodes.items():
        
        if len(childs) != 0:
            num_points = len(childs)
            theta = 2 * math.pi / num_points
            node_x, node_y = radius * math.cos(theta), radius * math.sin(theta)
            ax.plot(theta, radius, 'o')  # Plot node
            ax.text(theta, radius, str(node), ha='center', va='center', fontsize=8)
            # Draw edges
            
            for child in childs:
                angle = math.atan2(int(child[1]), int(child[0]))  # Calculate angle from child to center
                child_radius = math.sqrt(int(child[0])**2 + int(child[1])**2)  # Calculate radius from child to center
                ax.plot([theta, angle], [radius, child_radius], 'k-', linewidth=0.5)

    plt.show()

# Your existing code for node positions and child nodes

# Example usage
center_x = 0  # Example center x-coordinate
center_y = 0  # Example center y-coordinate
plot_radial_layout(sorted_node_childs, center_x, center_y)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the new center coordinates (theta, r)
center_theta = np.pi / 4  # Example angle (in radians)
center_r = 4.0  # Example radial distance

# Generate some sample data
#theta = np.linspace(0, 2 * np.pi, 100)
#r = np.sin(2 * theta)

# Plot the data with the new center
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

# Shift the center of the polar plot
ax.set_theta_zero_location('N')  # Set the zero angle to North (upward)
ax.set_theta_direction(-1)  # Set the direction of increasing angle counter-clockwise
ax.plot(theta + center_theta, r, color='blue')  # Plot the data with the new center
ax.scatter(center_theta, center_r, color='red')  # Plot the new center point
ax.set_rmax(center_r + 1)  # Adjust radial limits if necessary

plt.show()


In [ ]:
#HOW TO PLOT A GENERAL GRAPH
for edge in edges:
    start_node = edge.get_source()
    end_node = edge.get_destination()
    start_pos = node_positions[start_node]
    end_pos = node_positions[end_node]
    position = str(str(edge.get_attributes())).rfind(':')
    if int(position) != -1:
        w = int(str(edge.get_attributes())[position+3:-2])
        if w != 1:
            w = 1+w/10
    else:
        w=1
    plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w) 

# Draw nodes
for node, pos in node_positions.items():
    plt.plot(pos[0], pos[1], 'ro',markersize=10)  # 'ro' is red circle
    plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')

# Set plot limits
#plt.xlim(-0.5, 3.5)
#plt.ylim(-1.5, 1.5)

# Turn off axis
#plt.axis('off')

plt.savefig('plots/plot.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#def cartesian_to_polar(x, y):
#    radius = np.sqrt(x**2 + y**2)
#    theta = np.arctan2(y, x)
#    return radius, theta

def plot_radial_layout(node_positions, node_childs):
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.spines['polar'].set_visible(False)
    
    #plot node and edges
    for node, (x, y) in node_positions.items():
        radius, theta = x, y
        ax.plot(theta, radius, 'o') #plot node
        ax.text(theta, radius, str(node), ha='center', va='center', fontsize=8)
        # Draw edges
        for child in node_childs[node]:
            child_x, child_y = node_positions[child]
            child_radius, child_theta = child_x, child_y
            ax.plot([theta, child_theta], [radius, child_radius], 'k-',linewidth =0.5)

    plt.show()

# Your existing code for node positions and child nodes

# Example usage
plot_radial_layout(node_positions, node_childs)


### Sort the nodes based on the number of childs

In [ ]:
sorted_node_child_count = sorted_dict = dict(sorted(node_childs.items(), key=lambda item: len(item[1]), reverse=True))

def plot_radial_layout (sorted_node_child_count, node_childs):
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_xticks([])
    ax.spines['polar'].set_visible(False)
    
    #plot node and edges
    for node, (x, y) in sorted_node_child_count.items():
        radius, theta = x, y
        ax.plot(theta, radius, 'o') #plot node
        ax.text(theta, radius, str(node), ha='center', va='center', fontsize=8)
        # Draw edges
        for child in node_childs[node]:
            child_x, child_y = sorted_node_child_count[child]
            child_radius, child_theta = child_x, child_y
            ax.plot([theta, child_theta], [radius, child_radius], 'k-',linewidth =0.5)

    plt.show()

# Your existing code for node positions and child nodes

# Example usage
plot_radial_layout (sorted_node_child_count, node_childs)

# General Code

In [ ]:
print(edges.get_values())
#for node in nodes:
 #   print("Node name:", node.get_name())
  #  print("Node label:", node.get_label())
   # print("Node attributes:", node.get_attributes())
    #print("Node shape:", node.get_shape())
    #print("Node style:", node.get_style())
    #print("Node position:", node.get_pos())
    
    
#for edge in edges:
 #   print("Source:", edge.get_source())
  #  print("Destination:", edge.get_destination())
    #print("Edge attributes:", edge.get_attributes())
    #position = str(str(edge.get_attributes())).rfind(':')
    #w = str(edge.get_attributes())[position+3:-2]
    #print(w)
    #print("Edge style:", edge.get_style())
    #print("Edge label:", edge.get_label())
    #print("Edge color:", edge.get_color())

# TRASH

In [ ]:
#HOW TO PLOT A GENERAL GRAPH
for edge in edges:
    start_node = edge.get_source()
    end_node = edge.get_destination()
    start_pos = node_positions_radial[start_node]
    end_pos = node_positions_radial[end_node]

    position = str(str(edge.get_attributes())).rfind(':')
    if int(position) != -1:
        w = int(str(edge.get_attributes())[position+3:-2])
        if w != 1:
            w = 1+w/10
    else:
        w=1
    plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w) 

# Draw nodes
for node, pos in node_positions_radial.items():
    plt.plot(pos[0], pos[1], 'ro',markersize=10)  # 'ro' is red circle
    plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')

plt.savefig('radial_plot.png')
plt.show()

In [ ]:
#IGNORE: TREE WITH NODES
node_positions = {node.get_name(): (0, 0) for node in nodes}
#node_positions['1'] = (0, 0)
lonely_nodes = 

for node in nodes:
    max_y = node_child_count[str(node.get_name())]-1
    
    parent_x = node_positions[str(node.get_name())][0]
    parent_y = node_positions[str(node.get_name())][1]
    if len(node_childs[str(node.get_name())]) == 1:
        
        child_x = parent_x + 2
        child_y = parent_y  # Assuming you want to keep the same y-coordinate
        node_positions[child] = (child_x, child_y)
    
    elif len(node_childs[str(node.get_name())]) == 0:
        
    else:
        for child in node_childs[str(node.get_name())]:
            child_x = parent_x + 2
            child_y = parent_y + max_y
            node_positions[child] = (child_x, child_y)
            max_y -= 2
    #print(max_distance)
            
            
print(node_positions) 

In [ ]:
#COUNT THE NUMBER OF EDGES OF EACH NODE
node_edge_count = {node.get_name(): 0 for node in nodes}

for edge in graph[0].get_edges():
    s = edge.get_source()
    d = edge.get_destination()
    if s in node_edge_count:
        node_edge_count[s] += 1
    if d in node_edge_count:
        node_edge_count[d] += 1

for node_name, edge_count in node_edge_count.items():
    print("Node:", node_name, ", Number of edges:", edge_count)

In [ ]:
#HOW TO CREATE THE DEFAULT LAYOUT

with open(dot_file_path, 'r') as dot_file:
    dot_data = dot_file.read()

# Create a graph object from the .dot data
graph = pydot.graph_from_dot_data(dot_data)

# Set the layout algorithm to "dot" (or any other supported layout type)
graph[0].set_graph_defaults(layout="dot")

# Render the graph with Graphviz
output_file_path = 'dot_layout_graph.png'
graph[0].write_png(output_file_path)

print("Graph with dot layout saved as:", output_file_path)

In [ ]:
#HOW TO CREATE THE RADIAL LAYOUT
graph[0].set_graph_defaults(layout="twopi")
output_file_path = 'radial_graph.png'
graph[0].write_png(output_file_path)